In [1]:
import re
import pandas as pd
import requests
import openpyxl
from time import sleep
from random import uniform
from tqdm import tqdm
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [3]:
def get_page(url):
    try:
        response = requests.get(url, headers=headers, timeout=35)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'lxml')
    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
        return None

In [29]:
def scrape_criminals(url):
    i = 0
    links = []
    while i < 14:
        link = f"{url}?PAGEN_1={i+1}"
        try:
            response = requests.get(link, headers=headers)
            if response.status_code == 200:
                i += 1
                links.append(link)
                print(link)
            else:
                break
        except requests.exceptions.RequestException as e:
            print(f"Error fetching {link}: {e}")
            break
    soup_links = []
    for link in links:
        soup_links.append(get_page(link))
        sleep(uniform(1, 2))
    criminals = []
    for i in range(len(soup_links)):
        criminals.append(soup_links[i].find_all("div", class_="sl-item"))
    return criminals, links, soup_links

In [125]:
def scrape_criminal_data(criminals, links, url):
    pattern = r"^(?:[^/]*\/){2}(.*)"
    pattern_base_url = r'^(https?://[^/]+)'
    df = pd.DataFrame(columns=['Title', 'Text', 'Image', 'Region', 'District'])
    for i in tqdm(range(len(criminals))):
        for j in range(len(criminals[i])):
            title = ""
            image_url = ""
            full_text = ""
            region = ""
            district = ""
            a_tag_reg_dist = criminals[i][j].find_all("a", href="/territory/")
            if len(a_tag_reg_dist) >= 2:
                region = a_tag_reg_dist[0].text
                district = a_tag_reg_dist[1].text
            if len(a_tag_reg_dist) == 1:
                district = a_tag_reg_dist[0].text
            title_div = criminals[i][j].find("div", class_="sl-item-title links")
            if title_div:
                a_tag = title_div.find("a")
                if a_tag:
                    href_criminal = a_tag
                    if href_criminal:
                        match = re.search(pattern, href_criminal['href'])
                        if match:
                            extracted_part = match.group(1)
                            full_path_to_criminal = f'{url}{extracted_part}'
                            criminal_html = get_page(full_path_to_criminal)
                            if criminal_html:
                                criminal_info_blocks = criminal_html.find("div", class_="news-detail")
                                img_tag = criminal_info_blocks.find("img")
                                match = re.match(pattern_base_url, links[0])
                                if match:
                                    base_url = match.group(1)
                                    if img_tag:
                                        image_url = f'{base_url}{img_tag.get("src")}'
                                title_element = criminal_info_blocks.find(['h1', 'h2', 'h3', 'h4'])
                                if title_element:
                                    title = title_element.text
                                text_element = criminal_info_blocks.find_all(
                                    string=lambda text: (text.parent.name not in ['script', 'style', 'head', 'h3']) 
                                    and len(text.strip()) > 20
                                )
                                full_text = "\n".join([p.strip() for p in text_element])
                                df.loc[len(df)] = title, full_text, image_url, region, district
            print(df)
            sleep(uniform(1, 2))
    return df

In [5]:
url = "https://fsin.gov.ru/criminal/"

In [30]:
criminals, links, soup_links = scrape_criminals(url)

https://fsin.gov.ru/criminal/?PAGEN_1=1
https://fsin.gov.ru/criminal/?PAGEN_1=2
https://fsin.gov.ru/criminal/?PAGEN_1=3
https://fsin.gov.ru/criminal/?PAGEN_1=4
https://fsin.gov.ru/criminal/?PAGEN_1=5
https://fsin.gov.ru/criminal/?PAGEN_1=6
https://fsin.gov.ru/criminal/?PAGEN_1=7
https://fsin.gov.ru/criminal/?PAGEN_1=8
https://fsin.gov.ru/criminal/?PAGEN_1=9
https://fsin.gov.ru/criminal/?PAGEN_1=10
https://fsin.gov.ru/criminal/?PAGEN_1=11
https://fsin.gov.ru/criminal/?PAGEN_1=12
https://fsin.gov.ru/criminal/?PAGEN_1=13
https://fsin.gov.ru/criminal/?PAGEN_1=14


In [126]:
df = scrape_criminal_data(criminals, links, url)

  0%|          | 0/14 [00:00<?, ?it/s]

                           Title  \
0  Ориентировка на Карпенко А.А.   

                                                Text  \
0  Карпенко Алексей Александрович, 28.12.1992 год...   

                                               Image  \
0  https://fsin.gov.ru/upload/iblock/6f6/xq9spfce...   

                        Region                       District  
0  Костромская область (УФСИН)  Центральный федеральный округ  
                                        Title  \
0               Ориентировка на Карпенко А.А.   
1  Ориентировка : Долгов Александр Евгеньевич   

                                                Text  \
0  Карпенко Алексей Александрович, 28.12.1992 год...   
1  Долгов Александр Евгеньевич, 25.05.1998\nЦентр...   

                                               Image  \
0  https://fsin.gov.ru/upload/iblock/6f6/xq9spfce...   
1  https://fsin.gov.ru/upload/iblock/71a/j1wdrzqv...   

                        Region                       District  
0  Костромская область 

  7%|▋         | 1/14 [00:37<08:11, 37.80s/it]

                                                Title  \
0                       Ориентировка на Карпенко А.А.   
1          Ориентировка : Долгов Александр Евгеньевич   
2            Ориентировка: Ударцев Станислав Юрьевич    
3       Ориентировка: Тутундаев Александр Васильевич    
4           Ориентировка: Пидасова Галина Алексеевна    
5            Ориентировка: Мадумаров Юрий Валерьевич    
6           Ориентировка: Лебедев Дмитрий Анатольевич   
7   Ориентировка: Козлова Зарина Григорьевна (Голе...   
8                 Ориентировка: Каитов Алий Заурович    
9              Ориентировка: Жуков Андрей Михайлович    
10       Ориентировка: Емельянова Гузалия Шамильевна    
11     Ориентировка: Дорошенко Валентин Григорьевич     
12        Ориентировка: Джафаров Вахаб Агаджафар оглы   
13      Ориентировка: Ведерников Вячеслав Евгеньевич    
14           Ориентировка: Васильев Артем Георгиевич    
15             Ориентировка: Батаев Мамед Мовладович    
16          Ориентировка: Байра

 14%|█▍        | 2/14 [01:15<07:35, 37.96s/it]

...

 86%|████████▌ | 12/14 [07:36<01:15, 37.73s/it]

                                             Title  \
0                    Ориентировка на Карпенко А.А.   
1       Ориентировка : Долгов Александр Евгеньевич   
2         Ориентировка: Ударцев Станислав Юрьевич    
3    Ориентировка: Тутундаев Александр Васильевич    
4        Ориентировка: Пидасова Галина Алексеевна    
..                                             ...   
236              Ориентировка на Кочановского Е.И.   
237              Ориентировка на Незнамова  В. А.    
238                         Ор-ка на Морозова А.М.   
239                  Ориентировка на Великого В.А.   
240                      Ориентировка на Пителя Н.   

                                                  Text  \
0    Карпенко Алексей Александрович, 28.12.1992 год...   
1    Долгов Александр Евгеньевич, 25.05.1998\nЦентр...   
2    Ударцев Станислав Юрьевич 05.08.1991 г.р.\nСиб...   
3    Тутундаев Александр Васильевич 26.05.1979 г.р....   
4    Пидасова Галина Алексеевна 15.07.1969 г.р\nСиб...   
.. 

 93%|█████████▎| 13/14 [08:13<00:37, 37.63s/it]

                                                 Title  \
0                        Ориентировка на Карпенко А.А.   
1           Ориентировка : Долгов Александр Евгеньевич   
2             Ориентировка: Ударцев Станислав Юрьевич    
3        Ориентировка: Тутундаев Александр Васильевич    
4            Ориентировка: Пидасова Галина Алексеевна    
..                                                 ...   
256          Ориентировка: Киреев Максим Владимирович.   
257        Ориентировка на Аббасова Гариб Гульага Ог,    
258  Ориентировка на Горбунову (Арабаджи) Ольгу Ник...   
259         Ориентировка на Привалова Романа Игоревича   
260                 Ориентировка: Лялина Анна Игоревна   

                                                  Text  \
0    Карпенко Алексей Александрович, 28.12.1992 год...   
1    Долгов Александр Евгеньевич, 25.05.1998\nЦентр...   
2    Ударцев Станислав Юрьевич 05.08.1991 г.р.\nСиб...   
3    Тутундаев Александр Васильевич 26.05.1979 г.р....   
4    Пидасова

100%|██████████| 14/14 [08:19<00:00, 35.70s/it]


In [127]:
df.to_excel('fsin.xlsx')

In [128]:
df.to_csv('fsin.csv')